In [1]:
import os
import pandas as pd

df_path = "data/250327_data_from_RMI_from_productionbuild_Q3-2024_label 1.xlsx"

ori_df = pd.read_excel(df_path)
df = ori_df.copy()
print(df.shape)
df.head()

(1783, 82)


,id,control_name,control_owner,have_control,control_freq,control_effective_date,risk_question_id,control_des,control_design,response_type,...,mitigation_target_final_score,mitigation_cost_support,mitigation_other_support,mitigation_target_date,mitigation_plan_start_date,mitigation_plan_end_date,mitigation_plan_processes,mitigation_plan_causes,mitigation_plan_external_organization,existing_control_external_organization
0,NaN,NaN,NaN,False,NaN,NaN,96383,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],NaN
1,NaN,NaN,NaN,False,NaN,NaN,96318,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],NaN
2,NaN,NaN,NaN,False,NaN,NaN,96321,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],NaN
3,NaN,NaN,NaN,False,NaN,NaN,96299,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],NaN
4,NaN,NaN,NaN,False,NaN,NaN,96300,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,[],[],[],NaN


In [2]:
file_name = os.path.basename(df_path)
date_stamp = file_name.split("_")[0]
date_stamp

'250327'

In [3]:
# calc risk score
# Select columns that start with 'financial' but do not end with '_note'
# riskscore_columns = df.loc[:, (df.columns.str.startswith('Financial') | df.columns.str.startswith('NonFinancial') | df.columns.str.startswith('Likelihood')) & ~df.columns.str.endswith('_notes') & ~df.columns.str.endswith('_justification')]
df = ori_df.copy()
riskscore_columns = (
    (
        df.columns.str.startswith("Financial")
        | df.columns.str.startswith("NonFinancial")
        | df.columns.str.startswith("Likelihood")
    )
    & ~df.columns.str.endswith("_notes")
    & ~df.columns.str.endswith("_justification")
)

riskscore_financial_columns = (
    df.columns.str.startswith("Financial")
    & ~df.columns.str.endswith("_notes")
    & ~df.columns.str.endswith("_justification")
)
riskscore_nonfinancial_columns = (
    df.columns.str.startswith("NonFinancial")
    & ~df.columns.str.endswith("_notes")
    & ~df.columns.str.endswith("_justification")
)
riskscore_impact_columns = (
    (df.columns.str.startswith("Financial") | df.columns.str.startswith("NonFinancial"))
    & ~df.columns.str.endswith("_notes")
    & ~df.columns.str.endswith("_justification")
)
riskscore_likelihood_columns = (
    df.columns.str.startswith("Likelihood")
    & ~df.columns.str.endswith("_notes")
    & ~df.columns.str.endswith("_justification")
)
riskscore = df.loc[:, riskscore_columns]
# riskscore
import numpy as np

# Convert 'L1' to numeric value 1
# df[riskscore_columns] = df[riskscore_columns].str.extract('(\d+)').astype(int)


# Custom function to convert 'L1' to 1, handling missing values
def convert_to_numeric(val):
    if pd.isna(val):  # Check if the value is missing (NaN)
        return np.nan
    return int(val[1:])  # Remove the first character and convert to integer


# Apply the conversion to specific columns
# columns_to_convert = ['column1', 'column2']  # List of columns to transform
# columns_to_convert = df.loc[:,riskscore_columns].columns.tolist()
# # df.iloc[:,columns_to_convert] = df.iloc[:,columns_to_convert].applymap(convert_to_numeric)
# df[columns_to_convert] = df[columns_to_convert].apply(convert_to_numeric)

# Apply the conversion based on the boolean list
for i, convert in enumerate(riskscore_columns):
    if convert:
        df.iloc[:, i] = df.iloc[:, i].apply(convert_to_numeric)

# TODO: choose which method to aggregate, currently use max likelihood, max impact level
# compile impact, likelihood scores
# impact_column_names = rmi_data_control.loc[:,riskscore_impact_columns].columns.tolist()
# df['impact_combined'] = df[impact_column_names].max(axis=1) # choose max impact across all 'impacts' assessed by the same person
# likelihood_column_names = rmi_data_control.loc[:,riskscore_likelihood_columns].columns.tolist()
# df['likelihood_combined'] = df[likelihood_column_names].max(axis=1)

likelihood_column_names = df.loc[:, riskscore_likelihood_columns].columns.tolist()
impact_column_names = df.loc[:, riskscore_impact_columns].columns.tolist()
impact_fin_column_names = df.loc[:, riskscore_financial_columns].columns.tolist()
impact_nonfin_column_names = df.loc[:, riskscore_nonfinancial_columns].columns.tolist()

df["likelihood_combined"] = df[likelihood_column_names].max(axis=1)
df["impact_combined"] = df[impact_column_names].max(
    axis=1
)  # choose max impact across all 'impacts' assessed by the same person
df["impact_fin_combined"] = df[impact_fin_column_names].max(
    axis=1
)  # choose max impact across all 'impacts' assessed by the same person
df["impact_nonfin_combined"] = df[impact_nonfin_column_names].max(
    axis=1
)  # choose max impact across all 'impacts' assessed by the same person

df["risk_score"] = df["impact_combined"] * df["likelihood_combined"]
df["risk_score_fin"] = df["impact_fin_combined"] * df["likelihood_combined"]
df["risk_score_nonfin"] = df["impact_nonfin_combined"] * df["likelihood_combined"]


# Define the function to determine the risk level
def determine_risk_level(row, type):
    if type == "fin":
        riskscore = "risk_score_fin"
        riskimpact = "impact_fin_combined"
    elif type == "nonfin":
        riskscore = "risk_score_nonfin"
        riskimpact = "impact_nonfin_combined"
    else:
        riskscore = "risk_score"
        riskimpact = "impact_combined"
    if row[riskscore] >= 20:
        return 4
    elif 10 <= row[riskscore] <= 16:
        return 3
    elif 4 <= row[riskscore] <= 9:
        return 2
    elif (0 < row[riskscore] < 4) or (row[riskscore] == 4 and row[riskimpact] == 2):
        return 1
    elif row[riskscore] == 0:
        return 0
    # else:
    #     return 'Undefined'


# Apply the function to each row in the DataFrame
df["risk_level"] = df.apply(determine_risk_level, type="combined", axis=1)
df["risk_level_fin"] = df.apply(determine_risk_level, type="fin", axis=1)
df["risk_level_nonfin"] = df.apply(determine_risk_level, type="nonfin", axis=1)

In [4]:
debug_df = df[df["company"] == "PCG"]
debug_df = debug_df[debug_df["risk"].str.lower() == "poor supply quality"]
debug_df[["risk", "risk_score", "likelihood_combined", "impact_combined"]]

,risk,risk_score,likelihood_combined,impact_combined
1180,Poor supply quality,4.0,1.0,4.0
1223,Poor supply quality,5.0,5.0,1.0
1296,Poor supply quality,6.0,3.0,2.0
1307,Poor supply quality,3.0,1.0,3.0
1328,Poor supply quality,4.0,1.0,4.0
1329,Poor supply quality,4.0,1.0,4.0
1404,Poor supply quality,3.0,3.0,1.0


In [5]:
df2 = df.groupby(["company", "risk"], as_index=False).agg(
    {
        "risk_desc": set,
        "risk_cat": "first",
        "iscustom": set,  #'first', # risk
        "company_sector": set,
        "risk_subcat": set,
        "control_name": set,
        "control_des": set,
        "rootcause": set,
        "rootcause_desc": set,
        "process": set,
        "process_desc": set,
        "risk_level": max,
        "likelihood_combined": max,
        "impact_combined": max,
        "impact_fin_combined": max,
        "impact_nonfin_combined": max,
    }
)
# TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)
df2["risk_score"] = df2["impact_combined"] * df2["likelihood_combined"]
df2["risk_score_fin"] = df2["impact_fin_combined"] * df2["likelihood_combined"]
df2["risk_score_nonfin"] = df2["impact_nonfin_combined"] * df2["likelihood_combined"]

df2["risk_level"] = df2.apply(determine_risk_level, type="combine", axis=1)
df2["risk_level_fin"] = df2.apply(determine_risk_level, type="fin", axis=1)
df2["risk_level_nonfin"] = df2.apply(determine_risk_level, type="nonfin", axis=1)

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50698/4140459192.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df2 = df.groupby(["company", "risk"], as_index=False).agg(


In [6]:
print(df2.shape)
df2.head()

(1186, 23)


,company,risk,risk_desc,risk_cat,iscustom,company_sector,risk_subcat,control_name,control_des,rootcause,...,risk_level,likelihood_combined,impact_combined,impact_fin_combined,impact_nonfin_combined,risk_score,risk_score_fin,risk_score_nonfin,risk_level_fin,risk_level_nonfin
0,ALL NOW,Ineffective IT Operations Management,{ระบบสารสนเทศ IT ไม่พร้อมรองรับการให้บริการในท...,Operational risk,{False},{Services},{IT},"{การพัฒนาความรู้ ความสามารถ, การติดตามความคืบห...",{การพัฒนาความรู้ความสามารถของบุคลากรและนักพัฒน...,{['1.ความพร้อมของบุคลากรและนักพัฒนาระบบมีจำนวน...,...,2.0,2.0,4.0,NaN,4.0,8.0,NaN,8.0,NaN,2.0
1,ALL NOW,Negative media coverage,{ข้อร้องเรียนจากการทำงาน การปฏิบัติที่ไม่สอดคล...,Strategic Risk,{False},{Services},{Social},"{การกำหนดบทลงโทษ, การกำหนดขั้นตอนและกระบวนการต...",{1. ทีมงาน GoSoft คอยติดตามข้อมูลบนสื่อ Social...,{['1. ลูกค้า หรือพนักงานขนส่ง ได้รับการปฏิบัติ...,...,2.0,3.0,3.0,NaN,3.0,9.0,NaN,9.0,NaN,2.0
2,ALL NOW,Outsourcing inefficiency,{ออลล์ นาว ใช้ Subcontractor 100% ในการให้บริก...,Operational risk,{False},{Services},{Operation},"{การบริหารจัดการต้นทุน, การวางแผนการปฏิบัติงาน...",{1. จัดทำ New Cost structure Model เพื่ออัพเดท...,{['การพึ่งพาผู้รับจ้างภายนอกมากเกินไปสำหรับงาน...,...,2.0,2.0,3.0,NaN,3.0,6.0,NaN,6.0,NaN,2.0
3,ALL NOW,Product safety concern,{ความปลอดภัยจากการนำรถ EV มาใช้ (การออกแบบที่เ...,Operational risk,{False},{Services},{Operation},{การตรวจสอบความพร้อม และสภาพรถขนส่ง},{1. การตรวจสอบสภาพรถประจำวัน 2. ระบบแจ้งเตือนส...,{['ความไม่สมบูรณ์ของแบตเตอรี่ หรือการใช้งานที่...,...,1.0,1.0,3.0,NaN,3.0,3.0,NaN,3.0,NaN,1.0
4,ALL NOW,Protest,{การร้องเรียน ประท้วง และนัดหยุดงานของพนักงานข...,Strategic Risk,{False},{Services},{Political},"{การกำหนดมาตรฐานการจ่ายเงิน และโครงสร้างราคา, ...",{1. กำหนดมาตรฐานในการจ่ายเงินให้พขร. และมาตรกา...,{['1. พนักงานจัดส่ง มีการเปรียบเทียบการทำงานแล...,...,2.0,2.0,2.0,NaN,2.0,4.0,NaN,4.0,NaN,2.0


In [7]:
df2.company.value_counts()

company
CPCRT                                 135
True Properties                        86
PCG                                    71
CPLI                                   71
Advance AQ                             69
True Lifestyle Retail                  65
TIDC                                   64
Altervim                               61
Thana Digital Life_Game                57
Ascend Commerce Group                  53
W&W_BPD                                52
CPPC                                   44
CPMC                                   42
Ascend Money                           39
Freewill                               38
KPI/KSP                                38
Thana Digital Life_True Click Life     21
W&W_CCTS                               20
Advance-AHBG                           20
Chia Tai                               19
SLF                                    19
W&W_IT                                 18
W&W_DSRO                               15
ALL NOW                   

In [8]:
df3 = df2[df2["company"].isin(["PCG", "Advance AQ"])]
print(df3.shape)
df3.company.value_counts()

(140, 23)


company
PCG           71
Advance AQ    69
Name: count, dtype: int64

In [9]:
df3.head()

,company,risk,risk_desc,risk_cat,iscustom,company_sector,risk_subcat,control_name,control_des,rootcause,...,risk_level,likelihood_combined,impact_combined,impact_fin_combined,impact_nonfin_combined,risk_score,risk_score_fin,risk_score_nonfin,risk_level_fin,risk_level_nonfin
12,Advance AQ,After-sale service-related dissatisfaction,{ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้...,Operational risk,{False},{Pharmaceuticals},{Operation},{nan},{nan},{['ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคื...,...,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN
13,Advance AQ,Air pollution,{-},Strategic Risk,{False},{Pharmaceuticals},{Environment},{การตรวจวัดสิ่งแวดล้อม},{จัดให้มีการตรวจวัดค่าฝุ่นในบรรยากาศการทำงานแล...,{['ฝุ่นจากกระบวนการผลิต']},...,2.0,3.0,3.0,1.0,3.0,9.0,3.0,9.0,1.0,2.0
14,Advance AQ,Animal or insect infestation,{มีผลต่อคุณภาพในการจัดเก็บสินค้า},Strategic Risk,{False},{Pharmaceuticals},{Environment},{nan},{nan},{['วิธีการและเทคโนโลยีการควบคุมศัตรูพืชที่ไม่เ...,...,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
15,Advance AQ,Business interruption from fire hazards,{-},Operational risk,{False},{Pharmaceuticals},{Operation},{การฝึกอบรมพนักงานความปลอดภัยจากอัคคีภัย},{อบรมการดับเพลิงขั้นและเข้มงวดในการทำ 5ส.},{['ความประมาทหรือข้อผิดพลาดของพนักงานในการจัดก...,...,1.0,1.0,3.0,2.0,3.0,3.0,2.0,3.0,1.0,1.0
16,Advance AQ,Business interruption from flood,"{-, น้ำท่วมคลังสินค้า และเส้นทางขนส่ง}",Operational risk,{False},{Pharmaceuticals},{Operation},"{แผนการปิดปฏิบัติการชั่วคราวระหว่างน้ำท่วม, มี...",{ในกรณีที่น้ำท่วมสูงระดับต่างๆ บริษัทมีทีมรับผ...,{['เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การ...,...,2.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0,2.0,2.0


In [10]:
df3.columns

Index(['company', 'risk', 'risk_desc', 'risk_cat', 'iscustom',
       'company_sector', 'risk_subcat', 'control_name', 'control_des',
       'rootcause', 'rootcause_desc', 'process', 'process_desc', 'risk_level',
       'likelihood_combined', 'impact_combined', 'impact_fin_combined',
       'impact_nonfin_combined', 'risk_score', 'risk_score_fin',
       'risk_score_nonfin', 'risk_level_fin', 'risk_level_nonfin'],
      dtype='object')

In [11]:
select_columns = [
    "company",
    "risk_cat",
    "risk",
    "risk_desc",
    "rootcause",
    "rootcause_desc",
    "process",
    "process_desc",
    "risk_level",
]
df4 = df3[select_columns]
print(df4.shape)
df4.head()

(140, 9)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
12,Advance AQ,Operational risk,After-sale service-related dissatisfaction,{ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้...,{['ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคื...,{['ขั้นตอนการตรวจสอบการคืนสินค้าที่มีปัญหา หลา...,"{['Procurement', 'Research and Development']}",{['กระบวนการเคลมสินค้า กับ Supplier ต้องมีหลัก...,1.0
13,Advance AQ,Strategic Risk,Air pollution,{-},{['ฝุ่นจากกระบวนการผลิต']},{['ฝุ่นจากกระบวนการผลิต']},"{['Health, Safety, and Environment (HSE)', 'Pr...","{['', '']}",2.0
14,Advance AQ,Strategic Risk,Animal or insect infestation,{มีผลต่อคุณภาพในการจัดเก็บสินค้า},{['วิธีการและเทคโนโลยีการควบคุมศัตรูพืชที่ไม่เ...,{['']},"{['Quality Assurance', 'Warehouse']}","{['', '']}",1.0
15,Advance AQ,Operational risk,Business interruption from fire hazards,{-},{['ความประมาทหรือข้อผิดพลาดของพนักงานในการจัดก...,"{['', '']}","{['Health, Safety, and Environment (HSE)']}",{['']},1.0
16,Advance AQ,Operational risk,Business interruption from flood,"{-, น้ำท่วมคลังสินค้า และเส้นทางขนส่ง}",{['เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การ...,"{['', ''], ['เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้...","{['Other'], ['Warehouse', 'Transportation']}","{['', ''], ['การควบคุมดูแลเครื่องสูบน้ำของนิคม...",2.0


In [12]:
# select first row of column risk_desc
cell = df4.iloc[4, 4]
# cell = df4.iloc[2, 3]
# cell = df4.iloc[3, 3]
print(type(cell))
print(cell)
import json
import traceback
from typing import Union


def list_string_to_list(list_string) -> tuple[bool, Union[list, str]]:
    is_list = False
    if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
        (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
    ):

        return is_list, list_string
    # print(f"pass list_string :{type(list_string)} {list_string}")
    is_list = True
    # swap "'" to '"' and swap '"' to "'"
    list_string = list_string.replace('"', "###")
    list_string = list_string.replace("'", '"')
    list_string = list_string.replace("###", "'")
    try:
        list_cell = json.loads(list_string)
    except:
        is_list = False
        # print(f"here list_string : {list_string}")
        # print(traceback.format_exc())
        return is_list, list_string
    return is_list, list_cell


def cell_to_list(cell):
    empty_element_list = ["", "-"]
    if type(cell) == set:
        new_cell = []
        for list_string_candidate in cell:
            if list_string_candidate in empty_element_list:
                continue
            try:
                # if list_string count "'" or '"' have to be even and not 0
                # print(f"list_string_candidate : >>>{list_string_candidate}<<<")

                is_list, new_list_string_candidate = list_string_to_list(
                    list_string_candidate
                )
                # print(new_list_string_candidate == list_string_candidate)
                # print("is_list", is_list)
                # print(f"new_list_string_candidate : >>>{new_list_string_candidate}<<<")
                if is_list is False:
                    new_cell.append(new_list_string_candidate)
                    continue
                for i in new_list_string_candidate:
                    if i in empty_element_list:
                        continue

                    is_list, i = list_string_to_list(i)
                    # print(f"i : {is_list}>>>{i}<<<")
                    if is_list is False:
                        new_cell.append(i)
                        continue
                    new_cell.extend(i)

            except:
                # print(f"list_string : {list_string}")
                # print(traceback.format_exc())
                new_cell.append(list_string)
        return new_cell
    return cell


result = cell_to_list(cell)
print(cell_to_list(cell))
print(type(cell_to_list(cell)))

# print(cell_to_list(cell)[0])
# print(type(cell_to_list(cell)[0]))

<class 'set'>
{"['เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระบายน้ำล่าช้า']", "['ขาดเทคโนโลยีกันน้ำและป้องกันน้ำท่วม', 'ไม่มีแผนรับมือน้ำท่วม']"}
['เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระบายน้ำล่าช้า', 'ขาดเทคโนโลยีกันน้ำและป้องกันน้ำท่วม', 'ไม่มีแผนรับมือน้ำท่วม']
<class 'list'>


In [13]:
result

['เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระบายน้ำล่าช้า',
 'ขาดเทคโนโลยีกันน้ำและป้องกันน้ำท่วม',
 'ไม่มีแผนรับมือน้ำท่วม']

In [14]:
list_string = "ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า"
if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
    (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
):
    print("yes")

In [15]:
# df5 is df4 with pass each cell with cell_to_list
# convert set of list_string to list
df5 = df4.applymap(cell_to_list)
df5.head()

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50698/139688506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df5 = df4.applymap(cell_to_list)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
12,Advance AQ,Operational risk,After-sale service-related dissatisfaction,[ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้...,[ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคืนเ...,[ขั้นตอนการตรวจสอบการคืนสินค้าที่มีปัญหา หลายข...,"[Procurement, Research and Development]",[กระบวนการเคลมสินค้า กับ Supplier ต้องมีหลักฐา...,1.0
13,Advance AQ,Strategic Risk,Air pollution,[],[ฝุ่นจากกระบวนการผลิต],[ฝุ่นจากกระบวนการผลิต],"[Health, Safety, and Environment (HSE), Produc...",[],2.0
14,Advance AQ,Strategic Risk,Animal or insect infestation,[มีผลต่อคุณภาพในการจัดเก็บสินค้า],[วิธีการและเทคโนโลยีการควบคุมศัตรูพืชที่ไม่เหม...,[],"[Quality Assurance, Warehouse]",[],1.0
15,Advance AQ,Operational risk,Business interruption from fire hazards,[],[ความประมาทหรือข้อผิดพลาดของพนักงานในการจัดการ...,[],"[Health, Safety, and Environment (HSE)]",[],1.0
16,Advance AQ,Operational risk,Business interruption from flood,[น้ำท่วมคลังสินค้า และเส้นทางขนส่ง],[เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระ...,[เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระ...,"[Other, Warehouse, Transportation]",[การควบคุมดูแลเครื่องสูบน้ำของนิคมอุตสาหกรรม],2.0


In [16]:
# df6 convert list to string with join
df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)
df6.head()

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50698/3479902738.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)


,company,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
12,Advance AQ,Operational risk,After-sale service-related dissatisfaction,ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้น...,ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคืนเง...,ขั้นตอนการตรวจสอบการคืนสินค้าที่มีปัญหา หลายขั...,"Procurement,Research and Development",กระบวนการเคลมสินค้า กับ Supplier ต้องมีหลักฐาน...,1.0
13,Advance AQ,Strategic Risk,Air pollution,,ฝุ่นจากกระบวนการผลิต,ฝุ่นจากกระบวนการผลิต,"Health, Safety, and Environment (HSE),Production",,2.0
14,Advance AQ,Strategic Risk,Animal or insect infestation,มีผลต่อคุณภาพในการจัดเก็บสินค้า,วิธีการและเทคโนโลยีการควบคุมศัตรูพืชที่ไม่เหมาะสม,,"Quality Assurance,Warehouse",,1.0
15,Advance AQ,Operational risk,Business interruption from fire hazards,,ความประมาทหรือข้อผิดพลาดของพนักงานในการจัดการก...,,"Health, Safety, and Environment (HSE)",,1.0
16,Advance AQ,Operational risk,Business interruption from flood,น้ำท่วมคลังสินค้า และเส้นทางขนส่ง,เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระบ...,เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ทำให้การระบ...,"Other,Warehouse,Transportation",การควบคุมดูแลเครื่องสูบน้ำของนิคมอุตสาหกรรม,2.0


In [17]:
df7 = df6.copy()


# new_rootcause is 'rootcause : ' + rootcause column +\n+ 'rootcause_desc : ' rootcause_desc if it not '' empty string
# fundction to apply
def merge_rootcause(row):
    rootcause = row["rootcause"]
    rootcause_desc = row["rootcause_desc"]
    if rootcause == "" and rootcause_desc == "":
        return ""
    elif rootcause == "":
        return "rootcause_desc :" + rootcause_desc
    elif rootcause_desc == "":
        return "rootcause :" + rootcause
    else:
        return "rootcause :" + rootcause + "\n" + "rootcause_desc :" + rootcause_desc


df7["new_rootcause"] = df7.apply(merge_rootcause, axis=1)


def merge_process(row):
    process = row["process"]
    process_desc = row["process_desc"]
    if process == "" and process_desc == "":
        return ""
    elif process == "":
        return "process_desc :" + process_desc
    elif process_desc == "":
        return "process :" + process
    else:
        return "process :" + process + "\n" + "process_desc :" + process_desc


df7["new_process"] = df7.apply(merge_process, axis=1)
# remove rootcause and process column
df7.drop(
    ["rootcause", "rootcause_desc", "process", "process_desc"], axis=1, inplace=True
)
# rename new_rootcause and new_process to rootcause and process
df7.rename(
    columns={"new_rootcause": "rootcause", "new_process": "process"}, inplace=True
)
df7.head()

,company,risk_cat,risk,risk_desc,risk_level,rootcause,process
12,Advance AQ,Operational risk,After-sale service-related dissatisfaction,ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้น...,1.0,rootcause :ขั้นตอนการคืนสินค้า การแลกเปลี่ยน ห...,"process :Procurement,Research and Development\..."
13,Advance AQ,Strategic Risk,Air pollution,,2.0,rootcause :ฝุ่นจากกระบวนการผลิต\nrootcause_des...,"process :Health, Safety, and Environment (HSE)..."
14,Advance AQ,Strategic Risk,Animal or insect infestation,มีผลต่อคุณภาพในการจัดเก็บสินค้า,1.0,rootcause :วิธีการและเทคโนโลยีการควบคุมศัตรูพื...,"process :Quality Assurance,Warehouse"
15,Advance AQ,Operational risk,Business interruption from fire hazards,,1.0,rootcause :ความประมาทหรือข้อผิดพลาดของพนักงานใ...,"process :Health, Safety, and Environment (HSE)"
16,Advance AQ,Operational risk,Business interruption from flood,น้ำท่วมคลังสินค้า และเส้นทางขนส่ง,2.0,rootcause :เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ...,"process :Other,Warehouse,Transportation\nproce..."


In [18]:
risk_df_path = "data/20240811_ALL_risks.xlsx"
risk_df = pd.read_excel(risk_df_path, sheet_name="Risks")
selected_cols = [
    "Risk-EN",
    "Description-EN",
]
print(risk_df.columns)
risk_and_risk_desc_df = risk_df[selected_cols]
# apply strip to all columns
risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())

risk_and_risk_desc_df.head()
# create map_dict
risk_to_desc_dict = risk_and_risk_desc_df.set_index("Risk-EN")[
    "Description-EN"
].to_dict()
len(risk_to_desc_dict)

Index(['#', 'Risk category', 'Risk subcategory', 'Risk-EN', 'Description-EN',
       'Root cause-EN', 'Risk-TH', 'Description-TH', 'Root cause-TH',
       'Grouping index'],
      dtype='object')


/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_50698/3868739843.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())


165

In [19]:
df8 = df7.copy()
# print number cell == NaN
print(df8.isnull().sum().sum())
# risk column map with risk_to_desc_dict
# apply strip to df8["risk"]
df8["risk"] = df8["risk"].str.strip()
df8["risk_desc_catalog"] = df8["risk"].map(risk_to_desc_dict)

df8.head()
# is nan in risk_desc_catalog
print(df8["risk_desc_catalog"].isnull().sum())
# fill risk_desc_catalog nan with ""
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].fillna("")
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].str.strip()
# print row that risk_desc_catalog is nan
df8[df8["risk_desc_catalog"].isnull()]
# concat risk_desc_catalog and risk_desc
df8["risk_desc"] = df8["risk_desc_catalog"] + " " + df8["risk_desc"]
# strip df8["risk_desc"]
df8["risk_desc"] = df8["risk_desc"].str.strip()
# remove risk_desc_catalog
df8.drop("risk_desc_catalog", axis=1, inplace=True)
all_risk_cat = risk_df["Risk category"].unique().tolist()
# check is all df8['risk_cat'] in all_risk_cat
assert all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])
# add new data to df8 from risk_and_risk_cat_and_risk_desc_df
print(df8.shape)

new_rows = []
for index, row in risk_df.iterrows():
    risk_cat = row["Risk category"]
    risk_desc = row["Description-EN"]
    risk_name = row["Risk-EN"]
    risk_rootcause = row["Root cause-EN"]
    data = {
        # company	risk_cat	risk	risk_desc	rootcause	process
        "company": f"risk_catalog-{date_stamp}",
        "risk_cat": risk_cat,
        "risk": risk_name,
        "risk_desc": risk_desc,
        "rootcause": risk_rootcause,
        "process": "",
        "risk_level": 0,
    }
    new_rows.append(data)
df8 = pd.concat([df8, pd.DataFrame(new_rows)], ignore_index=True)
print(df8.shape)
print("nan", df8.isnull().sum().sum())

df8.head()

0
5
(140, 7)
(305, 7)
nan 0


,company,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,Advance AQ,Operational risk,After-sale service-related dissatisfaction,The risk of customers experiencing dissatisfac...,1.0,rootcause :ขั้นตอนการคืนสินค้า การแลกเปลี่ยน ห...,"process :Procurement,Research and Development\..."
1,Advance AQ,Strategic Risk,Air pollution,The presence of harmful substances in the air ...,2.0,rootcause :ฝุ่นจากกระบวนการผลิต\nrootcause_des...,"process :Health, Safety, and Environment (HSE)..."
2,Advance AQ,Strategic Risk,Animal or insect infestation,Risk of the invasion and proliferation of pest...,1.0,rootcause :วิธีการและเทคโนโลยีการควบคุมศัตรูพื...,"process :Quality Assurance,Warehouse"
3,Advance AQ,Operational risk,Business interruption from fire hazards,The business interruption due to a fire incide...,1.0,rootcause :ความประมาทหรือข้อผิดพลาดของพนักงานใ...,"process :Health, Safety, and Environment (HSE)"
4,Advance AQ,Operational risk,Business interruption from flood,The business interruption due to flooding in t...,2.0,rootcause :เครื่องสูบน้ำของนิคมชำรุดบ่อยครั้ง ...,"process :Other,Warehouse,Transportation\nproce..."


In [20]:
df.shape

(1783, 92)

In [21]:
df8.shape

(305, 7)

In [22]:
df_final = df8.copy()

In [23]:
# convert df6:dataframe to data_dict
result_list = df_final.to_dict(orient="records")
result_list[:3]

[{'company': 'Advance AQ',
  'risk_cat': 'Operational risk',
  'risk': 'After-sale service-related dissatisfaction',
  'risk_desc': 'The risk of customers experiencing dissatisfaction with the post-purchase support, assistance, or follow-up provided by the organization, leading to negative perceptions, reviews, or repurchase behavior. ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า',
  'risk_level': 1.0,
  'rootcause': 'rootcause :ขั้นตอนการคืนสินค้า การแลกเปลี่ยน หรือการคืนเงินที่ซับซ้อน ยืดเยื้อ หรือไม่มีประสิทธิภาพ ส่งผลกระทบต่อความสามารถของลูกค้าในการหาทางแก้ไขปัญหาที่เกี่ยวข้องกับผลิตภัณฑ์\nrootcause_desc :ขั้นตอนการตรวจสอบการคืนสินค้าที่มีปัญหา หลายขั้นตอน และใช้เวลานาน',
  'process': 'process :Procurement,Research and Development\nprocess_desc :กระบวนการเคลมสินค้า กับ Supplier ต้องมีหลักฐานเอกสารประกอบ,ต้องทำการตรวจสอบจากหน่วยงาน R&D ก่อน'},
 {'company': 'Advance AQ',
  'risk_cat': 'Strategic Risk',
  'risk': 'Air pollution',
  'risk_desc': 'The

In [24]:
# save it as json
import os

dir_path = os.getcwd()
save_file_path = f"{dir_path}/result/250327-company_risk_data.json"
# save result_list as json
import json

with open(save_file_path, "w") as f:
    json.dump(result_list, f, ensure_ascii=False)